# Person Re-identification using PySDK
This notebook demonstrates Person Re-Identification (Re-ID) using PySDK. Re-ID focuses on recognizing and matching people across different camera views based on their unique appearance, like clothing and body shape.

The basic pipeline works like this:
1. Detect people in the image using a person detection model.
2. Crop each detected person using the bounding box coordinates.
3. Apply the Person Re-ID model to the cropped images to extract the embeddings which can further be used to identify and match individuals across different images or camera views.

In [ ]:
import degirum as dg, degirum_tools

inference_host_address = "@local"
zoo_url = "degirum/hailo"
token = '' 
device_type = "HAILORT/HAILO8L"

# Person detection model name 
person_det_model_name = "yolov8n_relu6_person--640x640_quant_hailort_hailo8l_1"

# load AI model
person_det_model = dg.load_model(
    model_name=person_det_model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type
    
)

# Choose the Person reid model name 
person_reid_model_name = "osnet_x1_0_person_reid--256x128_quant_hailort_hailo8l_1"
# person_reid_model_name = "repvgg_a0_person_reid--256x128_quant_hailort_hailo8l_1" 

# load AI model
person_reid_model = dg.load_model(
    model_name=person_reid_model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type
    
)

#### Let's walk through a practical example of person re-identification.<br> 

We have three input images:

- **Two images of the same individual** (Monica)
- **One image of a different individual** (Phoebe)

The goal is to verify whether the model can correctly identify that the first two images belong to the same person and distinguish the third image as a different person.

In [ ]:
#Image sources
image_source_monica_1 = "../assets/Friends_dataset/Monica_2.jpg"
image_source_monica_2 = "../assets/Friends_dataset/Monica_4.png"
image_source_phoebe = "../assets/Friends_dataset/Phoebe_4.png"

# Detections
detections_monica_1 = person_det_model(image_source_monica_1)
detections_monica_2 = person_det_model(image_source_monica_2)
detections_phoebe = person_det_model(image_source_phoebe)

In [ ]:
# Utility function for displaying crops

import matplotlib.pyplot as plt

def display_images(images, titles="Images", figsize=(15, 5)):
    """
    Display a list of images in a single row using Matplotlib.
    
    Parameters:
    - images (list): List of images (NumPy arrays) to display.
    - titles (str or list): Either a single string for overall title, or list of titles for each image.
    - figsize (tuple): Size of the figure.
    """
    num_images = len(images)
    fig, axes = plt.subplots(1, num_images, figsize=figsize)
    if num_images == 1:
        axes = [axes]  # Make iterable for single image

    for i, (ax, image) in enumerate(zip(axes, images)):
        image_rgb = image[:, :, ::-1]  # Convert BGR to RGB
        ax.imshow(image_rgb)
        ax.axis('off')
        if isinstance(titles, list) and i < len(titles):
            ax.set_title(titles[i], fontsize=12)

    if isinstance(titles, str):
        fig.suptitle(titles, fontsize=16)

    plt.tight_layout()
    plt.show()

In [ ]:
# Cropping
x1, y1, x2, y2 = map(int, detections_monica_1.results[0]["bbox"])  # Convert bbox coordinates to integers
monica_crop_1 = detections_monica_1.image[y1:y2, x1:x2]  # Crop the person from the image

x1, y1, x2, y2 = map(int, detections_monica_2.results[0]["bbox"])  # Convert bbox coordinates to integers
monica_crop_2 = detections_monica_2.image[y1:y2, x1:x2]  # Crop the person from the image

x1, y1, x2, y2 = map(int, detections_phoebe.results[0]["bbox"])  # Convert bbox coordinates to integers
phoebe_crop = detections_phoebe.image[y1:y2, x1:x2]  # Crop the person from the image

# Display person crops
display_images([monica_crop_1, monica_crop_2, phoebe_crop], titles=["Monica1","Monica2","Phoebe"], figsize=(10, 5))

### Extracting embedding using a Person recognition model for each person crop

In [ ]:
#Extract the embeddings
monica_embedding_1 = person_reid_model(monica_crop_1).results[0]["data"][0][0] # shape (1,512)
monica_embedding_2 = person_reid_model(monica_crop_2).results[0]["data"][0][0] # shape (1,512)
phoebe_embedding = person_reid_model(phoebe_crop).results[0]["data"][0][0] # shape (1,512)

### Calculating cosine similarity between the embeddings

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity between two images of Monica
similarity_1 = cosine_similarity(monica_embedding_1, monica_embedding_2)
print("Cosine similarity between two images of Monica: ", similarity_1[0][0])

# Compute cosine similarity between Monica and Phoebe
similarity_2 = cosine_similarity(monica_embedding_1, phoebe_embedding)
print("Cosine similarity between Monica and Phoebe: ", similarity_2[0][0])

### The results show that the two images of Monica are the same person as the cosine similarity is larger than the one compared between two images of two different people.